In [1]:
from pymongo import MongoClient
import pprint

In [2]:
client = MongoClient('mongodb://localhost:27017')

In [3]:
import json

In [37]:
database = client['Twitter']
db = database['tweets']

In [17]:
data = []
with open('tweets.json', 'rb') as f:
    for line in f:
        data.append(json.loads(line))

In [18]:
print len(data)

51428


In [38]:
for t in data:
    try: 
        db.insert_one(t)
    except:
        continue

In [39]:
db.find_one()

{u'_id': ObjectId('573c8d7dde301b0cf4b0c146'),
 u'contributors': None,
 u'coordinates': None,
 u'created_at': u'Thu Sep 02 18:11:23 +0000 2010',
 u'entities': {u'hashtags': [], u'urls': [], u'user_mentions': []},
 u'favorited': False,
 u'geo': None,
 u'id': 22819396900L,
 u'in_reply_to_screen_name': None,
 u'in_reply_to_status_id': None,
 u'in_reply_to_user_id': None,
 u'place': None,
 u'retweet_count': None,
 u'retweeted': False,
 u'source': u'web',
 u'text': u'eu preciso de terminar de fazer a minha tabela, est\xe1 muito foda **',
 u'truncated': False,
 u'user': {u'contributors_enabled': False,
  u'created_at': u'Fri Jul 03 21:44:05 +0000 2009',
  u'description': u's\xf3 os loucos sabem (:',
  u'favourites_count': 1,
  u'follow_request_sent': None,
  u'followers_count': 102,
  u'following': None,
  u'friends_count': 73,
  u'geo_enabled': False,
  u'id': 53507833,
  u'lang': u'en',
  u'listed_count': 0,
  u'location': u'',
  u'name': u'Beatriz Helena Cunha',
  u'notifications': None,


In [30]:
def most_tweets():
    result = db.aggregate([
            {'$group':{'_id':'$user.screen_name',
                        'count':{'$sum':1}}},
            {'$sort':{'count':-1}} ])
    return result

In [44]:
#result = most_tweets()
#for d in result:
    #pprint.pprint(d)

In [ ]:
"""
The tweets in our twitter collection have a field called "source". This field describes the application
that was used to create the tweet. Following the examples for using the $group operator, your task is 
to modify the 'make-pipeline' function to identify most used applications for creating tweets. 
As a check on your query, 'web' is listed as the most frequently used application.
'Ubertwitter' is the second most used. The number of counts should be stored in a field named 'count'
(see the assertion at the end of the script). """

In [ ]:
def make_pipeline():
    # complete the aggregation pipeline
    pipeline = [{'$group':{'_id':'$source',
                           'count':{'$sum':1}}},
                {'$sort': {'count':-1}}
               ]
    return pipeline

In [ ]:
#$project:shapes documents for next stage such as subsetting fields
#$match: filters documents based on specified criteria
#$skip: allows skip over a certain number of documents at beginning
#$limit: limits query to top specified number of documents
#$unwind: unpacks array fields and creates an instance for each document with unpacked area field:value pairs

In [60]:
def most_follows():
    result = db.aggregate([
            {'$project':{'_id':0, 'screen_name':'$user.screen_name', 'followers':'$user.followers_count'}},
            {'$sort':{'followers':-1}},
            {'$limit':10}
        ])
    for a in result:
        pprint.pprint(a)
most_follows()

{u'followers': 854199, u'screen_name': u'garyvee'}
{u'followers': 457722, u'screen_name': u'jimmycarr'}
{u'followers': 330179, u'screen_name': u'lilduval'}
{u'followers': 330151, u'screen_name': u'lilduval'}
{u'followers': 307902, u'screen_name': u'Adela_Micha'}
{u'followers': 289006, u'screen_name': u'fitrop'}
{u'followers': 275708, u'screen_name': u'GuyKawasaki'}
{u'followers': 259760, u'screen_name': u'otaviomesquita'}
{u'followers': 208325, u'screen_name': u'fatbellybella'}
{u'followers': 199501, u'screen_name': u'JackAllTimeLow'}


In [55]:
def highest_ratio():
    result = db.aggregate([
            {'$match':{'user.friends_count':{'$gt':0}, #limit to users with friends, followers > 0
                    'user.followers_count':{'$gt':0}}},
            {'$project':{'ratio':{'$divide':['$user.followers_count', #ratio for followers:friends
                                              '$user.friends_count']},
                         "screen_name":'$user.screen_name'}},
            {'$sort':{'ratio':-1}}, #sort in descending order
            {'$limit':10}   #limit top result            
        ])
    return result

In [56]:
result = highest_ratio()
for t in result:
    print t

{u'_id': ObjectId('573c8d8bde301b0cf4b14580'), u'ratio': 19221.5, u'screen_name': u'Twitterrific'}
{u'_id': ObjectId('573c8d8ade301b0cf4b13a3f'), u'ratio': 17124.0, u'screen_name': u'steve_berra'}
{u'_id': ObjectId('573c8d8ade301b0cf4b13cf0'), u'ratio': 16847.0, u'screen_name': u'2dopeboyz'}
{u'_id': ObjectId('573c8d8ede301b0cf4b1630a'), u'ratio': 13894.222222222223, u'screen_name': u'backstreetboys'}
{u'_id': ObjectId('573c8d81de301b0cf4b0e394'), u'ratio': 13155.666666666666, u'screen_name': u'tedouumdado'}
{u'_id': ObjectId('573c8d81de301b0cf4b0e589'), u'ratio': 10962.0, u'screen_name': u'3in1of2'}
{u'_id': ObjectId('573c8d85de301b0cf4b10969'), u'ratio': 8742.0, u'screen_name': u'craigslistjobs'}
{u'_id': ObjectId('573c8d8ede301b0cf4b15e0c'), u'ratio': 8720.0, u'screen_name': u'LaJornada'}
{u'_id': ObjectId('573c8d86de301b0cf4b11b2f'), u'ratio': 5918.0, u'screen_name': u'trampos'}
{u'_id': ObjectId('573c8d92de301b0cf4b18769'), u'ratio': 3661.0, u'screen_name': u'StyleCareers'}


In [ ]:
"""
Write an aggregation query to answer this question:

Of the users in the "Brasilia" timezone who have tweeted 100 times or more,
who has the largest number of followers?

The following hints will help you solve this problem:
- Time zone is found in the "time_zone" field of the user object in each tweet.
- The number of tweets for each user is found in the "statuses_count" field.
  To access these fields you will need to use dot notation (from Lesson 4)
- Your aggregation query should return something like the following:
{u'ok': 1.0,
 u'result': [{u'_id': ObjectId('52fd2490bac3fa1975477702'),
                  u'followers': 2597,
                  u'screen_name': u'marbles',
                  u'tweets': 12334}]}
Note that you will need to create the fields 'followers', 'screen_name' and 'tweets'."""

In [63]:
def brasilia():
    result = db.aggregate([
            {'$match':{'user.time_zone':'Brasilia', #limit to users in Brasilia
                    'user.statuses_count':{'$gte':100}}}, #tweeted 100 times or more
            {'$project':{'followers': '$user.followers_count',
                         "screen_name":'$user.screen_name',
                        'tweets': '$user.statuses_count'}},
            {'$sort':{'followers':-1}}, #sort in descending order
            {'$limit':1}   #limit top result            
        ])
    for a in result:
        pprint.pprint(a)
brasilia()

{u'_id': ObjectId('573c8d82de301b0cf4b0ee80'),
 u'followers': 259760,
 u'screen_name': u'otaviomesquita',
 u'tweets': 10997}


In [ ]:
# db.find({'entities.user_mentions':{'$size':3}}) #documents with user_mentions of length 3

In [64]:
def user_mentions():
    results = db.aggregate([
            {'$unwind': '$entities.user_mentions'},
            {'$group': {'_id':'$user.screen_name',
                       'count': {'$sum': 1}}},
            {'$sort':{'count':-1}},
            {'$limit': 1}
        ])
    for a in results:
        pprint.pprint(a)
user_mentions()

{u'_id': u'ThizBoySwagLoud', u'count': 21}


In [ ]:
"""
For this exercise, let's return to our cities infobox dataset. The question we would like you to answer
is as follows:  Which region or district in India contains the most cities? (Make sure that the count of
cities is stored in a field named 'count'; see the assertions at the end of the script.) """

In [65]:
def india():
    [    {'$match': {'country': 'India'}}, #limits to India
            {'$unwind': '$isPartOf'},
            {'$group': {'_id':'$isPartOf',
                       'count': {'$sum': 1}}},
            {'$sort':{'count':-1}},
            {'$limit': 1}
        ]

In [66]:
def hashtag_retweet_avg():
    results = db.aggregate([
            {'$unwind': '$entities.hashtags'},
            {'$group': {'_id':'$entities.hashtags.text',
                       'retweet_avg': {'$avg': '$retweet_count'}}},
            {'$sort':{'retweet_avg':-1}},
            {'$limit': 10}
        ])
    for a in results:
        pprint.pprint(a)
hashtag_retweet_avg()

{u'_id': u'actress', u'retweet_avg': None}
{u'_id': u'love', u'retweet_avg': None}
{u'_id': u'wordpress', u'retweet_avg': None}
{u'_id': u'amigu\xeds', u'retweet_avg': None}
{u'_id': u'Thingsthatmakeyougohmmmm', u'retweet_avg': None}
{u'_id': u'movies', u'retweet_avg': None}
{u'_id': u'SinCableTVRecomienda', u'retweet_avg': None}
{u'_id': u'Michiles2233', u'retweet_avg': None}
{u'_id': u'NEWS', u'retweet_avg': None}
{u'_id': u'littleknownfactsaboutme', u'retweet_avg': None}


In [67]:
def unique_hashtags_by_user():
    results = db.aggregate([
            {'$unwind': '$entities.hashtags'},
            {'$group': {'_id':'$user.screen_name',
                       'unique_hashtags': {'$addToSet': '$entities.hashtags.text'}}},
            {'$sort':{'_id':-1}}
        ])

In [71]:
"""
$push is similar to $addToSet. The difference is that rather than accumulating only unique values 
it aggregates all values into an array.

Using an aggregation query, count the number of tweets for each user. In the same $group stage, 
use $push to accumulate all the tweet texts for each user. Limit your output to the 5 users
with the most tweets. 
Your result documents should include only the fields:
"_id" (screen name of user), 
"count" (number of tweets found for the user),
"tweet_texts" (a list of the tweet texts found for the user).  
"""

def tweet_counts():
    results = db.aggregate([
            {'$group': {'_id':'$user.screen_name',
                       'count': {'$sum': 1},
                       'tweet_texts': {'$push': '$text'}}},
            {'$sort':{'count':-1}},
            {'$limit': 5}
        ])
    #for r in results:
     #   pprint.pprint(r)
    return results
tweet_counts()

In [73]:
def unique_user_mentions():
    results = db.aggregate([
            {'$unwind': '$entities.user_mentions'},
            {'$group': {'_id':'$user.screen_name',
                       'mset': {'$addToSet': '$entities.user_mentions.screen_name'}}},
            {'$unwind': '$mset'},
            {'$group':{'_id':'$_id', 'count': {'$sum':1}}},
            {'$sort':{'count':-1}},
            {'$limit': 10}
        ])
    for a in results:
        pprint.pprint(a)
unique_user_mentions()

{u'_id': u'Democracy_Work', u'count': 17}
{u'_id': u'ThizBoySwagLoud', u'count': 16}
{u'_id': u'itsajstuerd', u'count': 15}
{u'_id': u'FollowersNeeded', u'count': 15}
{u'_id': u'Egreeedy', u'count': 12}
{u'_id': u'Spirics', u'count': 12}
{u'_id': u'soulbrother64', u'count': 11}
{u'_id': u'MULA_BSB', u'count': 11}
{u'_id': u'burnedoutdoc', u'count': 11}
{u'_id': u'thiagothamiel', u'count': 10}


In [74]:
"""
In an earlier exercise we looked at the cities dataset and asked which region in India contains 
the most cities. In this exercise, we'd like you to answer a related question regarding regions in 
India. What is the average city population for a region in India? Calculate your answer by first 
finding the average population of cities in each region and then by calculating the average of the 
regional averages.

Hint: If you want to accumulate using values from all input documents to a group stage, you may use 
a constant as the value of the "_id" field. For example, 
    { "$group" : {"_id" : "India Regional City Population Average",
      ... } """
def india_city():
    [    {'$match': {'country': 'India'}}, #limits to India
            {'$unwind': '$isPartOf'},
            {'$group': {'_id':'$isPartOf',
                       'average': {'$avg': '$population'}}},
            {'$group': {'_id': 'India Regional City Population Average',
                       'avg': {'$avg': '$average'}}}
        ]

In [3]:
"""
What is the most common city name in our cities collection?

Your first attempt probably identified None as the most frequently occurring
city name. What that actually means is that there are a number of cities
without a name field at all. 

To solve this problem the right way, we should really ignore cities that don't
have a name specified. As a hint ask yourself what pipeline operator allows us
to simply filter input? How do we test for the existence of a field?"""
def make_pipeline():
    pipeline = [ {'$match':{'name':{'$ne': None}}},
                 {'$group':{'_id':'$name', 'count':{'$sum':1}}},
                 {'$sort':{'count': -1}},
                 {'$limit': 1}
    ]
    return pipeline

In [11]:
"""
Which Region in India has the largest number of cities with longitude between
75 and 80?"""
def make_pipeline():
    pipeline = [ {'$match':{'lon':{'$gte': 75, '$lte': 80}, 'country':'India'}},
                 {'$unwind': '$isPartOf'},
                 {'$group': {'_id':'$isPartOf',
                       'count': {'$sum': 1}}},
                 {'$sort':{'count': -1}},
                 {'$limit': 1}
    ]
    return pipeline

In [12]:
"""
Extrapolating from an earlier exercise in this lesson, find the average
regional city population for all countries in the cities collection. What we
are asking here is that you first calculate the average city population for each
region in a country and then calculate the average of all the regional averages
for a country.

  As a hint, _id fields in group stages need not be single values. They can
also be compound keys (documents composed of multiple fields). You will use the
same aggregation operator in more than one stage in writing this aggregation
query. I encourage you to write it one stage at a time and test after writing
each stage. """
def make_pipeline():
    pipeline = [
            {'$unwind': '$isPartOf'}, #unwinds by region
            {'$group': {'_id':{'country':'$country', "isPartOf": '$isPartOf'}, #compound key with country and region
                       'average': {'$avg': '$population'}}}, #average by country/region key
            {'$group': {'_id': '$_id.country', #group again by country
                       'avgRegionalPopulation': {'$avg': '$average'}}} #take average over country
        ]
    return pipeline

In [ ]:
def elevation():
    pipeline = [
            {'$group': {'_id': '$country', 'avgElevation': {'$avg':'$elevation'}}},
            {'$sort': {'avgElevation':-1}}
        ]
    return pipeline